<a href="https://colab.research.google.com/github/2j0123/WASSUP_EST_FINAL_Team4/blob/SW/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive와 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요한 라이브러리 import
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# 데이터 경로 설정
base_dir = '/content/drive/MyDrive/감정따뜻쟁이/train'

# 이미지 데이터 생성기 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 이미지 데이터 로드
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

# Inception-v3 모델 불러오기 (사전 훈련된 가중치 포함)
base_model = InceptionV3(weights='imagenet', include_top=False)

# 분류기 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# 새로운 모델 정의 (기존의 Inception-v3 모델과 분류기를 결합)
model = Model(inputs=base_model.input, outputs=predictions)

# 모든 레이어를 동결하여 기존의 가중치를 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_generator, epochs=170)

# 새로운 이미지 분류 테스트
def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # 이미지 배치 추가
    img_array /= 255.  # 이미지 정규화
    predictions = model.predict(img_array)
    emotion_labels = ['anger', 'happy', 'sadness', 'panic']
    predicted_label = emotion_labels[np.argmax(predictions)]
    return predicted_label

# 예시: 이미지 분류 테스트
test_image_path = '/content/drive/MyDrive/test_image.jpg'  # 테스트할 이미지 파일의 경로
predicted_label = classify_image(test_image_path)
print('Predicted emotion:', predicted_label)

Found 5369 images belonging to 4 classes.
87910968/87910968 [==============================] - 3s 0us/step
Epoch 1/170
168/168 [==============================] - 2498s 15s/step - loss: 1.5416 - accuracy: 0.3198
Epoch 2/170
168/168 [==============================] - 404s 2s/step - loss: 1.3103 - accuracy: 0.3762
Epoch 3/170
168/168 [==============================] - 402s 2s/step - loss: 1.2916 - accuracy: 0.3863
Epoch 4/170
168/168 [==============================] - 405s 2s/step - loss: 1.2655 - accuracy: 0.4081
Epoch 5/170
168/168 [==============================] - 398s 2s/step - loss: 1.2473 - accuracy: 0.4308
Epoch 6/170
168/168 [==============================] - 394s 2s/step - loss: 1.2456 - accuracy: 0.4267
Epoch 7/170
168/168 [==============================] - 397s 2s/step - loss: 1.2212 - accuracy: 0.4411
Epoch 8/170
168/168 [==============================] - 395s 2s/step - loss: 1.2096 - accuracy: 0.4425
Epoch 9/170
168/168 [==============================] - 410s 2s/step - loss:

In [ ]:
# Google Colab에서 파일 업로드하기 위한 라이브러리 import
from google.colab import files

# 이미지 분류 함수 정의
def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # 이미지 배치 추가
    img_array /= 255.  # 이미지 정규화
    predictions = model.predict(img_array)
    emotion_labels = ['anger', 'happy', 'sadness', 'panic']
    predicted_label = emotion_labels[np.argmax(predictions)]
    return predicted_label

# 사용자가 업로드한 파일을 받아서 이미지 분류하는 함수 정의
def classify_uploaded_images():
    uploaded = files.upload()  # 사용자로부터 파일 업로드 받기
    for filename in uploaded.keys():
        image_path = '/content/' + filename
        predicted_label = classify_image(image_path)
        print(f'{filename}의 감정은 {predicted_label}입니다.')

# 사용자가 원하는 만큼 사진 또는 영상을 업로드하고 분류
while True:
    classify_uploaded_images()
    continue_upload = input("더 업로드하시겠습니까? (yes/no): ")
    if continue_upload.lower() != 'yes':
        break

Saving 0pnnbde1bec084d09efe5011eacbd99aa3ef8ad004d0d15ae3381adc39bd9j0qo.jpg to 0pnnbde1bec084d09efe5011eacbd99aa3ef8ad004d0d15ae3381adc39bd9j0qo.jpg
1/1 [==============================] - 0s 29ms/step
0pnnbde1bec084d09efe5011eacbd99aa3ef8ad004d0d15ae3381adc39bd9j0qo.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving okss9bb8ceb2bbef180fbee6d54ecefdd331d67f7fe3bb167c03ddeb5d890go7q.jpg to okss9bb8ceb2bbef180fbee6d54ecefdd331d67f7fe3bb167c03ddeb5d890go7q.jpg
1/1 [==============================] - 0s 27ms/step
okss9bb8ceb2bbef180fbee6d54ecefdd331d67f7fe3bb167c03ddeb5d890go7q.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): no


In [ ]:
# 필요한 라이브러리 import
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from google.colab import files
import cv2
import os

# 데이터 경로 설정 (훈련 및 검증 데이터)
base_dir = '/content/drive/MyDrive/감정따뜻쟁이/img/train'
validation_dir = '/content/drive/MyDrive/감정따뜻쟁이/img/val'

# 이미지 데이터 생성기 설정 (훈련 및 검증 데이터)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 데이터 로드 (훈련 및 검증 데이터)
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Inception-v3 모델 불러오기 (사전 훈련된 가중치 포함)
base_model = InceptionV3(weights='imagenet', include_top=False)

# 분류기 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# 새로운 모델 정의 (기존의 Inception-v3 모델과 분류기를 결합)
model = Model(inputs=base_model.input, outputs=predictions)

# 모든 레이어를 동결하여 기존의 가중치를 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_generator, epochs=170, validation_data=validation_generator)

# 검증 데이터에 대한 예측
validation_generator.reset()
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes

# 성능 지표 출력
target_names = list(validation_generator.class_indices.keys())
print(classification_report(y_true, y_pred_classes, target_names=target_names))

# 혼동 행렬 출력
print(confusion_matrix(y_true, y_pred_classes))

# 새로운 이미지 분류 테스트 함수
def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # 이미지 배치 추가
    img_array /= 255.  # 이미지 정규화
    predictions = model.predict(img_array)
    emotion_labels = ['anger', 'happy', 'sadness', 'panic']
    predicted_label = emotion_labels[np.argmax(predictions)]
    return predicted_label

# 사용자가 업로드한 파일을 받아서 이미지 분류하는 함수 정의
def classify_uploaded_images():
    uploaded = files.upload()  # 사용자로부터 파일 업로드 받기
    for filename in uploaded.keys():
        image_path = '/content/' + filename
        predicted_label = classify_image(image_path)
        print(f'{filename}의 감정은 {predicted_label}입니다.')

# 사용자가 원하는 만큼 사진 또는 영상을 업로드하고 분류
while True:
    classify_uploaded_images()
    continue_upload = input("더 업로드하시겠습니까? (yes/no): ")
    if continue_upload.lower() != 'yes':
        break

Found 5366 images belonging to 4 classes.
Found 1049 images belonging to 5 classes.
Epoch 1/170
168/168 [==============================] - ETA: 0s - loss: 1.5422 - accuracy: 0.3189

InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <la

In [ ]:
# 필요한 라이브러리 import
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

# 데이터 경로 설정 (훈련 및 검증 데이터)
base_dir = '/content/drive/MyDrive/감정따뜻쟁이/img/train'
validation_dir = '/content/drive/MyDrive/감정따뜻쟁이/img/val'
test_dir = '/content/drive/MyDrive/감정따뜻쟁이/TEST_DATA_SET/TEST_DATA_SET'

# 이미지 데이터 생성기 설정 (훈련 및 검증 데이터)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 데이터 로드 (훈련 및 검증 데이터)
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Inception-v3 모델 불러오기 (사전 훈련된 가중치 포함)
base_model = InceptionV3(weights='imagenet', include_top=False)

# 분류기 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# 새로운 모델 정의 (기존의 Inception-v3 모델과 분류기를 결합)
model = Model(inputs=base_model.input, outputs=predictions)

# 모든 레이어를 동결하여 기존의 가중치를 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_generator, epochs=170, validation_data=validation_generator)

# 검증 데이터에 대한 예측
validation_generator.reset()
y_val_pred = model.predict(validation_generator)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)
y_val_true = validation_generator.classes

# 성능 지표 출력 (검증 데이터)
val_target_names = list(validation_generator.class_indices.keys())
print("Validation Data Performance:")
print(classification_report(y_val_true, y_val_pred_classes, target_names=val_target_names))
print(confusion_matrix(y_val_true, y_val_pred_classes))

# 테스트 데이터에 대한 예측
test_generator.reset()
y_test_pred = model.predict(test_generator)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true = test_generator.classes

# 성능 지표 출력 (테스트 데이터)
test_target_names = list(test_generator.class_indices.keys())
print("Test Data Performance:")
print(classification_report(y_test_true, y_test_pred_classes, target_names=test_target_names))
print(confusion_matrix(y_test_true, y_test_pred_classes))


Found 5366 images belonging to 4 classes.
Found 1049 images belonging to 4 classes.
Found 1013 images belonging to 4 classes.
87910968/87910968 [==============================] - 3s 0us/step
Epoch 1/170
168/168 [==============================] - 3150s 18s/step - loss: 1.5252 - accuracy: 0.3185 - val_loss: 1.3440 - val_accuracy: 0.2974
Epoch 2/170
168/168 [==============================] - 466s 3s/step - loss: 1.3001 - accuracy: 0.3848 - val_loss: 1.3137 - val_accuracy: 0.3775
Epoch 3/170
168/168 [==============================] - 461s 3s/step - loss: 1.2855 - accuracy: 0.3899 - val_loss: 1.3008 - val_accuracy: 0.3651
Epoch 4/170
168/168 [==============================] - 459s 3s/step - loss: 1.2558 - accuracy: 0.4133 - val_loss: 1.3354 - val_accuracy: 0.3613
Epoch 5/170
168/168 [==============================] - 462s 3s/step - loss: 1.2568 - accuracy: 0.4193 - val_loss: 1.2820 - val_accuracy: 0.4051
Epoch 6/170
168/168 [==============================] - 466s 3s/step - loss: 1.2418 - ac